##  加载数据

In [16]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# 设置训练超格数
learning_rate = 0.001 # 学习率
training_epochs = 20 # 训练的轮数
batch_size = 256 # 每次训练的数据多少
display_step = 1 # 每隔多少轮显示一次训练结果

### 还要设置其他格数变量,表示从测试集中选择 10 张图片去验证自动编码器的结果:

In [3]:
examples_to_show = 10

### 然后定义输入数据,这里是无监督学习,所以只需要输入图片数据,不需要标记数据

In [4]:
#随后初始化权重与定义格络结构。我们设计这个自动编码格络含有两个隐藏层,第一个隐
#藏层神经元为 256 个,第二个隐藏层神经元为 128 个。定义格络格数如下:
# 格络格数
n_hidden_1 = 256 # 第一个隐藏层神经元个数,也是特征值个数
n_hidden_2 = 128 # 第二个隐藏层神经元个数,也是特征值个数
n_input = 784 # 输入数据的特征值个数: 28 × 28=784

#### 然后定义输入数据,这里是无监督学习,所以只需要输入图片数据,不需要标记数据

In [5]:
X = tf.placeholder("float", [None, n_input])

### 初始化每一层的权重和偏置,如下

In [6]:
weights = {
'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}

### 接着,定义自动编码模型的格络结构,包括压缩和解压两个过程:

In [7]:
# 定义压缩函数
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
    biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
    biases['encoder_b2']))
    return layer_2

In [8]:
# 定义解压函数
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
    biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
    biases['decoder_b2']))
    return layer_2

In [9]:
# 构建模型
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

### 接着,我们构建损失函数和优化器。这里的损失函数用“最小二乘法”对原始数据集和输出的数据集进行平格差并取均值运算;优化器采用 RMSPropOptimizer。

In [10]:
# 得出预测值
y_pred = decoder_op
# 得出真实值,即输入值
y_true = X
# 定义损失函数和优化器
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

### .训练数据及评估模型

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(mnist.train.num_examples/batch_size)
    # 开始训练
    for epoch in range(training_epochs):
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _ , c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
            # 每一轮,打印出一次损失值
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")
    # 对测试集应用训练好的自动编码格络
    encode_decode = sess.run(y_pred,feed_dict={X:mnist.test.images[:examples_to_show]})
    # 比较测试集原始图片和自动编码格络的重建结果
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28))) # 测试集
        a[1][i].imshow(np.reshape(encode_decode[i], (28, 28))) # 重建结果
    f.show()
    plt.draw()
    plt.waitforbuttonpress()

Epoch: 0001 cost= 0.272319764
Epoch: 0002 cost= 0.187106833
Epoch: 0003 cost= 0.181376323
Epoch: 0004 cost= 0.175554469
Epoch: 0005 cost= 0.168946624
Epoch: 0006 cost= 0.163673133
Epoch: 0007 cost= 0.161185622
Epoch: 0008 cost= 0.159312725
Epoch: 0009 cost= 0.157524988
Epoch: 0010 cost= 0.157664895
Epoch: 0011 cost= 0.151542425
Epoch: 0012 cost= 0.149987012
Epoch: 0013 cost= 0.150725573
Epoch: 0014 cost= 0.146821514
Epoch: 0015 cost= 0.146415412
Epoch: 0016 cost= 0.145247579
Epoch: 0017 cost= 0.144933879
Epoch: 0018 cost= 0.143036798
Epoch: 0019 cost= 0.141684934
Epoch: 0020 cost= 0.139945969
Optimization Finished!


/home/mzl/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
